In [13]:
### PREPARATION ###

import pandas as pd
import numpy as np

#Loading Datasets

Axis = pd.read_csv (r'/home/kaustubh/Downloads/Axis_Bank_BSE.csv')
n_50 = pd.read_csv (r'/home/kaustubh/Downloads/NIFTY_50.csv')
n_bank = pd.read_csv (r'/home/kaustubh/Downloads/NIFTY_BANK.csv')

#Renaming Columns and Dropping Date, Volume and Close Columns 

n_50.rename(columns = {'Date':'N50_Date', 'Open':'N50_Open', 'High':'N50_High', 'Low':'N50_Low', 'Close':'N50_Close', 'Adj Close': 'N50_Adj Close', 'Volume': 'N50_Volume'}, inplace = True)
n_bank.rename(columns = {'Date':'NB_Date','Open':'NB_Open', 'High':'NB_High', 'Low':'NB_Low', 'Close':'NB_Close', 'Adj Close': 'NB_Adj Close', 'Volume': 'NB_Volume'}, inplace = True)

Axis = Axis.drop(['Date', 'Close'], axis = 1)
Nifty_50 = n_50.drop(['N50_Date', 'N50_Volume', 'N50_Close', 'N50_Open', 'N50_High', 'N50_Low'], axis=1)
Nifty_Bank = n_bank.drop(['NB_Date', 'NB_Volume', 'NB_Close', 'NB_Open', 'NB_High', 'NB_Low'], axis=1)

#Merging the datasets and deleting rows with any null values to create Project Dataset

Axis_Working = pd.concat([Axis, Nifty_50, Nifty_Bank], axis = 1)
Axis_Working = Axis_Working.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

Axis_Working.reset_index(drop=True, inplace=True)

Axis_Working_C = Axis_Working.copy()
Axis_Working_C["Sign"] =  Axis_Working_C["Adj Close"].diff(1)
Axis_Working_C.iat[0, 7] = 0
Axis_Working_C["Direction"] = ""

i=0
for j in range(2164):

    if(Axis_Working_C.iat[i, 7] >= 0):
        Axis_Working_C.iat[i, 8] = 1
    else:
        Axis_Working_C.iat[i, 8]= 0
    i = i + 1
    
Axis_Working_C = Axis_Working_C.drop(['Sign'], axis = 1)

#Creating Train and Test sets for Regression

Axis_Working_Train = Axis_Working[ : 1732]
Axis_Working_Test = Axis_Working[1732 : ]

#Creating Train and Test sets for Classifiaction

Axis_Working_C_train=Axis_Working_C[ : 1732]
Axis_Working_C_test=Axis_Working_C[1732 : ]

In [5]:
Axis_Working_C.head()

,Open,High,Low,Adj Close,Volume,N50_Adj Close,NB_Adj Close
0,247.199997,248.389999,243.210007,226.918472,612515.0,6157.600098,11855.750000
1,246.000000,246.600006,237.000000,220.039368,762995.0,6146.350098,11564.049805
2,237.600006,238.600006,229.800003,214.120529,998500.0,6079.799805,11305.450195
3,232.000000,238.800003,230.399994,215.302460,730150.0,6048.250000,11186.799805
4,235.000000,245.779999,232.639999,225.671906,929630.0,5904.600098,11053.349609


In [6]:
Axis_Working_C.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2165 entries, 0 to 2164
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           2165 non-null   float64
 1   High           2165 non-null   float64
 2   Low            2165 non-null   float64
 3   Adj Close      2165 non-null   float64
 4   Volume         2165 non-null   float64
 5   N50_Adj Close  2165 non-null   float64
 6   NB_Adj Close   2165 non-null   float64
dtypes: float64(7)
memory usage: 118.5 KB


In [17]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

Axis_Targets = Axis_Working_Train["Adj Close"].copy()
Axis_Linear =  Axis_Working_Train.drop("Adj Close", axis=1)

lin_reg = LinearRegression()

#Cross Validation

scores = cross_val_score(lin_reg, Axis_Linear, Axis_Targets, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

Scores: [5.67719245 2.74899922 2.46593308 4.63984033 2.95341284 6.10087208
 4.315554   3.28850385 4.29090367 6.80757693]
Mean: 4.328878845341406
Standard deviation: 1.419191455889225


In [9]:
Axis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2461 non-null   float64
 1   High       2461 non-null   float64
 2   Low        2461 non-null   float64
 3   Adj Close  2461 non-null   float64
 4   Volume     2461 non-null   float64
dtypes: float64(5)
memory usage: 96.6 KB


In [10]:
Axis_Working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2165 entries, 0 to 2164
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           2165 non-null   float64
 1   High           2165 non-null   float64
 2   Low            2165 non-null   float64
 3   Adj Close      2165 non-null   float64
 4   Volume         2165 non-null   float64
 5   N50_Adj Close  2165 non-null   float64
 6   NB_Adj Close   2165 non-null   float64
dtypes: float64(7)
memory usage: 118.5 KB


In [18]:
###Decision Tree Regressor##$

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


Axis_Targets = Axis_Working_Train["Adj Close"].copy()
Axis_DT =  Axis_Working_Train.drop("Adj Close", axis=1)

tree_reg = DecisionTreeRegressor()

#Cross Validation

scores = cross_val_score(tree_reg, Axis_DT, Axis_Targets, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

Scores: [ 6.90915326  5.13714622  5.55129909  5.79909855 25.89479053  9.84305529
  7.71602385  7.80129557  7.32491528 73.35816043]
Mean: 15.533493806685646
Standard deviation: 20.121073712890226


In [20]:
###Random Forest Regressor###

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

Axis_Targets = Axis_Working_Train["Adj Close"].copy()
Axis_RF =  Axis_Working_Train.drop("Adj Close", axis=1)

forest_reg = RandomForestRegressor()

#Cross Validation

scores = cross_val_score(forest_reg, Axis_RF, Axis_Targets, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

#Though the scores here are better than Decision Tree, they are worse than Linear Regression. 
#Is it so because the Random Forest Regressor is still overfitting the data?

Scores: [ 6.15210479  3.64281096  3.18717494  4.98156889 11.44903474  8.59548429
  5.73689062  4.34777773  6.43176426 72.90406254]
Mean: 12.742867377348265
Standard deviation: 20.18770330224721


In [21]:
### SVR ###

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

Axis_Targets = Axis_Working_Train["Adj Close"].copy()
Axis_SVR =  Axis_Working_Train.drop("Adj Close", axis=1)

svm_reg = SVR()


#Cross Validation

scores = cross_val_score(svm_reg, Axis_SVR, Axis_Targets, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

#No idea why SVR is overfitting the data so badly (even worse than HDFC)

Scores: [227.89463587 254.09190649 221.71238611 221.61436592 151.22959813
 203.85158255 205.45407475 206.69067555 238.31293787 362.89517724]
Mean: 229.37473404679366
Standard deviation: 51.46311534860379
